# Momentum Trading Strategy Implementation

In [8]:
import yfinance as yf
import backtrader as bt
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

In [24]:
class Momentum(bt.Indicator):
    # every trading day has a momentum parameter
    # except for the first 90 days
    lines = ('momentum_trend',)
    params = (('period', 90),)
    
    def __init__(self):
        self.addminperiod = self.params.period
        
    def next(self):
        returns = np.log(self.data.get(size=self.params.period))
        x = np.arange(len(returns))
        beta, _, rvalue, _, _ = linregress(x, returns)
        annualized = (1 + beta) ** 252
        self.lines.momentum_trend[0] = annualized * rvalue ** 2

In [26]:
class MomentumStrategy(bt.Strategy):
    
    def __init__(self):
        self.counter = 0
        self.indicators = {}
        self.sorted_data = []
        # we store the SP500 (this is the index) data as the first item of the dataset
        self.spy = self.data[0]
        # all the other stocks (present in SP500)
        self.stocks = self.data[1:]
        
        for stock in stocks:
            self.indicators[stock] = {}
            self.indicators[stock]['momentum'] = Momentum(stock.close, period = 90)
            self.indicators[stock]['sma100'] = bt.indicators.\
                MovingAverageSimple(stock.close, period=100)
            self.indicators[stock]['atr20'] = bt.indicators.ATR(stock, period=20)
            
        # SMA for SP500 index - because we open long positions when the SP500 index
        # is above its SMA(200) BULLISH MARKET
        self.sma200 = bt.indicators.MovingAverageSimple(self.spy.close, period=200)

In [22]:
def download_stocks(stocks, start, end):
    data = {}
    
    for stock in stocks:
        ticker = yf.download(stock, start, end)
        data[stock] = ticker['Adj Close']
        
    return pd.DataFrame(data)    

In [23]:
def calculate_momentum(data):
    log_data = np.log(data)
    x_data = np.arange(len(log_data))
    beta, _, rvalue, _, _ = linregress(x_data, log_data)
    # We have to annualize the slope
    # There are 252 trading days in a year
    return (1+beta)**252 * (rvalue**2)